In [ ]:
########now part 2: decode and train#######

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18044654973256815325
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11272650752
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3002063844608420621
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import timeit
import matplotlib.pyplot as plt
#from preprocess_func_new import *
from matplotlib.image import imread
import os
from os.path import expanduser
%matplotlib inline


#tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
img_path = os.path.join('..','datasets','stixels')
img_path

'../datasets/stixels'

In [5]:
batch_size=8
val_batch_size=batch_size
test_batch_size=batch_size

In [6]:
tfrec_train_directory = os.path.join('..','datasets','stixels','train','tfrec_batch_size_'+str(batch_size))

In [7]:
#only for showing pics with predictions later
import os
sum_csv_test_path = os.path.join(img_path,'test', 'sum_csv')
labels_test=pd.read_csv(os.path.join(sum_csv_test_path,'labels_test.csv'))
test_names_list=list(labels_test['Name'])
image_paths_test=[]
for name in test_names_list:
    image_paths_test.append(os.path.join(img_path, 'test', name+'.png')) #maybe no need to add '.png'


In [8]:
path_tfrecords_train_lst=[]
path_tfrecords_train = os.path.join(img_path, 'train')
for root, dirs, files in os.walk(tfrec_train_directory):
    for file in files:
        if '.tfrecord' in file:
            path_tfrecords_train_lst.append(os.path.join(tfrec_train_directory,file))
        else:
            print('WARNING: file ' + file + 'looks suspicious. does it belong here?')



In [9]:
tfrec_val_directory = os.path.join('..','datasets','stixels','val','tfrec_batch_size_'+str(val_batch_size))

In [10]:
path_tfrecords_val_lst=[]
path_tfrecords_val = os.path.join(img_path, 'val')
for root, dirs, files in os.walk(tfrec_val_directory):
    for file in files:
        if '.tfrecord' in file:
            path_tfrecords_val_lst.append(os.path.join(tfrec_val_directory,file))
        else:
            print('WARNING: file ' + file + 'looks suspicious. does it belong here?')

In [11]:
H=370 
W=24
C=3
img_shape = (H, W, C)
num_classes = 47

In [12]:
def parse(serialized):
    # Define a dict with the data-names and types we expect to
    # find in the TFRecords file.
    # It is a bit awkward that this needs to be specified again,
    # because it could have been written in the header of the
    # TFRecords file instead.
    features = \
        {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        }

    # Parse the serialized data so we get a dict with our data.
    parsed_example = tf.parse_single_example(serialized=serialized,
                                             features=features)

    # Get the image as raw bytes.
    image_raw = parsed_example['image']

    # Decode the raw bytes so it becomes a tensor with type.
    #######image = tf.decode_raw(image_raw, tf.int32) ####
    image = tf.image.decode_png(image_raw, channels=3, dtype=tf.uint16)
    #image = tf.cast(image, tf.int32)

    # The type is now uint8 but we need it to be float.
    
    image = tf.cast(image, tf.float32) ####
    
    # Get the label associated with the image.
    label = parsed_example['label']

    # The image and label are now correct TensorFlow types.
    return image, label

In [13]:
def input_fn(filenames, train, batch_size=batch_size, buffer_size=40000): #2048
    # Args:
    # filenames:   Filenames for the TFRecords files.
    # train:       Boolean whether training (True) or testing (False).
    # batch_size:  Return batches of this size.
    # buffer_size: Read buffers of this size. The random shuffling
    #              is done on the buffer, so it must be big enough.

    # Create a TensorFlow Dataset-object which has functionality
    # for reading and shuffling data from TFRecords files.
    dataset = tf.data.TFRecordDataset(filenames=filenames)

    # Parse the serialized data in the TFRecords files.
    # This returns TensorFlow tensors for the image and labels.
    dataset = dataset.map(parse)

    if train:
        # If training then read a buffer of the given size and
        # randomly shuffle it.
        dataset = dataset.shuffle(buffer_size=buffer_size)

        # Allow infinite reading of the data.
        num_repeat = None
    else:
        # If testing then don't shuffle the data.
        
        # Only go through the data once.
        num_repeat = 1

    # Repeat the dataset the given number of times.
    dataset = dataset.repeat(num_repeat)
    
    # Get a batch of data with the given size.
    dataset = dataset.batch(batch_size)

    # Create an iterator for the dataset and the above modifications.
    iterator = dataset.make_one_shot_iterator()

    # Get the next batch of images and labels.
    images_batch, labels_batch = iterator.get_next()

    # The input-function must return a dict wrapping the images.
    x = {'image': images_batch}
    y = labels_batch
    
    return x, y

In [14]:
def train_input_fn():
    return input_fn(filenames=path_tfrecords_train_lst, train=True)

In [15]:
def val_input_fn():
    return input_fn(filenames=path_tfrecords_val_lst, train=False)

In [16]:
def test_input_fn():
    return input_fn(filenames=path_tfrecords_test, train=False)

In [17]:
#for pred later only
def load_images(image_paths):
    # Load the images from disk.
    images = [imread(path) for path in image_paths]
    # Convert to a numpy array and return it.
    return np.asarray(images)


In [18]:
#for pred later only ##TODO:SHUFFLE!
some_num=10
some_images = load_images(image_paths=image_paths_test[0:some_num])
some_images_cls = np.array(labels_test['Label'])

In [19]:
some_images_cls

array([46, 46, 46, ..., 46, 46, 46])

In [20]:
#check if this works TODO
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"image": some_images.astype(np.float32)},
    num_epochs=1,
    shuffle=False)

In [21]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "image" in the input-function.
    x = features["image"]
    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1,W,H,C])
    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=64, kernel_size=(11,5),
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(8,4), strides=1)
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=200, kernel_size=(5,3),
                           padding='same', activation=tf.nn.relu) #200
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(4,3), strides=1)    

    # Flatten to a 2-rank tensor.
    #net = tf.contrib.layers.flatten(net)
    # Eventually this should be replaced with:
    net = tf.layers.flatten(net)

    # First fully-connected / dense layer.
    # This uses the ReLU activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=512, activation=tf.nn.relu)     #1024
    
    # Second fully-connected / dense layer
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=512, activation=tf.nn.relu)   #2048
    
   
    # This is the last layer so it does not use an activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc3',
                          units=47)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        metrics = \
        {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

In [22]:
params = {"learning_rate": 1e-4}

In [23]:
def make_dir_and_comment(model_dir):
    home = expanduser("~")
    log_name=os.path.join('logs/', model_dir + '.txt')
    
    if os.path.isdir(model_dir):
        print('INFO: dir with name ' + model_dir + ' already exist.')
    
    new_comment=input('Please add a comment\n')
    
    if os.path.exists(log_name):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    
    model_log = open(log_name,append_write)
    model_log.write(home +' : '+ new_comment + '\n')
    model_log.close()
    return



In [24]:
model_dir = './ckpts_19_5_lenet_fc_512_512_b8' #'./ckpts_<day>_<month>_<architecture>_<main_change>'
make_dir_and_comment(model_dir) 
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir=model_dir)


INFO: dir with name ./ckpts_19_5_lenet_fc_512_512 already exist.
Please add a comment
testing val
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_task_type': 'worker', '_save_checkpoints_secs': 600, '_is_chief': True, '_model_dir': './ckpts_19_5_lenet_fc_512_512', '_tf_random_seed': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe11df88860>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_global_id_in_cluster': 0, '_master': '', '_task_id': 0}


In [49]:
model.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./ckpts_15_5_lenet_fc_512_512/model.ckpt.
INFO:tensorflow:loss = 24578.582, step = 1
INFO:tensorflow:global_step/sec: 0.499859
INFO:tensorflow:loss = 65.05324, step = 101 (200.058 sec)
INFO:tensorflow:global_step/sec: 0.509998
INFO:tensorflow:loss = 10.715708, step = 201 (196.079 sec)
INFO:tensorflow:Saving checkpoints for 288 into ./ckpts_15_5_lenet_fc_512_512/model.ckpt.
INFO:tensorflow:global_step/sec: 0.436222
INFO:tensorflow:loss = 9.946377, step = 301 (229.241 sec)
INFO:tensorflow:global_step/sec: 0.507144
INFO:tensorflow:loss = 14.191336, step = 401 (197.183 sec)
INFO:tensorflow:global_step/sec: 0.506467
INFO:tensorflow:loss = 304.28192, step = 501 (197.446 sec)
INFO:tensorflow:Saving checkpoints for 

#####DONE TRAIN######

In [ ]:
# VAL 

In [25]:
val_result = model.evaluate(input_fn=val_input_fn) ##need to reduce batch size for val and test

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-20-07:47:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ckpts_19_5_lenet_fc_512_512/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-20-08:47:15
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.05045493, global_step = 1000, loss = 10.2984495


In [26]:
val_result

{'accuracy': 0.05045493, 'global_step': 1000, 'loss': 10.2984495}

In [ ]:
print("Classification val accuracy: {0:.2%}".format(val_result["accuracy"]))

In [ ]:
###test###

In [27]:
test_result = model.evaluate(input_fn=test_input_fn)

NameError: name 'path_tfrecords_test' is not defined

In [ ]:
test_result

In [ ]:
print("Classification test accuracy: {0:.2%}".format(test_result["accuracy"]))

In [ ]:
###pred###

In [ ]:
predictions = model.predict(input_fn=predict_input_fn) ###FIX THIS

In [ ]:
cls_pred = np.array(list(predictions))
cls_pred #FIX THIS